In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
class Simulator:
    def __init__(
        self, 
        num_genes = 20000, 
        avg_num_sgRNAs = 5, 
        num_treatment = 2, 
        num_control = 2, 
        min_total = 1000,
        max_total = 100000,
        total_NTCs = 1000,
        fraction_enriched = 0.2,
        fraction_depleted = 0.2,
        fraction_NTC = 0.2):
        
        """
        Constructor for initializing Simulator object.
        
        Parameters
        ----------
        num_genes : int
            Number of genes.
        avg_num_sgRNAs : int
            Average number of sgRNAs across all genes. 
        num_treatment : int
            Number of treatment of libraries.
        num_control : int
            Number of control of libraries. 
        min_total : int
            The lower bound of the total number of counts for one library. 
        max_total : int
            The upper bound of the total number of counts for one library.
        total_NTCs : int
            Total number of non-targeting controls.
        fraction_enriched : float
            The fraction of enriched genes with respect to all genes. 
        fraction_depleted : float
            The fraction of depleted genes with respect to all genes. 
        fraction_NTC : float
            The fraction of NTC genes with respect to all genes. 
            
        Raises
        ------
        Exception
            If the total of fractions (enriched, depleted, NTC) exceeds 1. 
        
        """
        self.num_genes = num_genes
        self.avg_num_sgRNAs = avg_num_sgRNAs
        self.num_treatment = num_treatment
        self.num_control = num_control
        self.min_total = min_total
        self.max_total = max_total
        self.total_NTCs = total_NTCs
        
        total_fractions = fraction_enriched + fraction_depleted + fraction_NTC
        
        if ((total_fractions > 0.0) & (total_fractions <= 1.0)):
            self.fraction_enriched = fraction_enriched
            self.fraction_depleted = fraction_depleted
            self.fraction_NTC = fraction_NTC
            self.fraction_normal = 1.0 - (fraction_enriched + fraction_depleted + fraction_NTC)
        else:
            raise Exception("Fractions total cannot exceed 1.")
        
        self.totals_array = np.random.randint(self.min_total, self.max_total, size = self.num_treatment + self.num_control)
        
        self.bounds = [10, 30]
        
        self.sgRNAs = self._num_sgRNAs()
        
        self.num_e = round(len(self.sgRNAs) * self.fraction_enriched)
        self.num_d = round(len(self.sgRNAs) * self.fraction_depleted)
        self.num_ntc = round(len(self.sgRNAs) * self.fraction_NTC)
        self.num_n = round(len(self.sgRNAs) * self.fraction_normal)
        
        self.lam = [np.random.uniform(self.bounds[0], self.bounds[1]) for gene in self.sgRNAs for num in np.arange(gene)]
        
        #make option of whether nbinomial or poisson, trucated normal 
    
    def _sgRNAs(self):
        return ["sgRNA_" + str(int(i)) for i in np.arange(self.sgRNAs.sum())]
    
    def _gene(self):
        """
        Generates list of numbered genes for use in sample() DataFrame. 
        
        Returns
        -------
        list
            The elements of the list are in numerical order up to the 
            number of genes in `num_genes`.
        
        """
        return ["gene_" + str(i) for i in np.arange(len(self.sgRNAs)) for n in np.arange(self.sgRNAs[i])]
    
    
    def _num_sgRNAs(self):
        """
        Generates a number of sgRNAs per gene. 
        
        Returns
        -------
        sgRNAs : array
            The values of the array follow a normal distrubution with the
            mean being `avg_num_sgRNAs`.
        
        """
        sgRNAs = np.random.normal(loc=self.avg_num_sgRNAs, scale=1, size=self.num_genes)
        sgRNAs = np.round(sgRNAs)
        return sgRNAs 
 
        
    def _sum_array(self, index, lambdas):
        """
        Creates an array of random integers with a specified sum.
        
        Parameters
        ----------
        index : int
            The index to specify which total to use from `totals_array` 
            defined in the constructor. 
            
        Returns
        -------
        a : array
            array of randomly generated integers with sum of element from `totals_array`    
        
        """
        
        a = [np.random.poisson(i, size=1) for i in lambdas]
        a = np.concatenate(a)
        a = a.astype(float)
        a /= (a.sum())
        a *= self.totals_array[index]
        a = np.round(a)
        
        return a
    
    def _setting_treatment_libraries(self):
        """
        Generates values for treatment libraries.
        
        Returns
        -------
        treatment : list
            `treatment` is a list of arrays, one for each library, 
            generated by the _sum_array() method. 
            
        """
        treatment = [] 
        
        for i in np.arange(self.num_treatment):
            treatment.append(self._sum_array(i, self._S_l()))
        
        return treatment
    
    def _setting_control_libraries(self):
        """
        Generates values for control libraries.
        
        Returns
        -------
        control : list
            `control` is a list of arrays, one for each library, 
            generated by the _sum_array() method. 
            
        """
        control = [] 
        
        for i in np.arange(self.num_control):
            control.append(self._sum_array(-(i+1), self.lam))
        
        return control


    def _g_e(self):
        return self.sgRNAs[0: self.num_e]      
    
    def _g_d(self):
        return self.sgRNAs[self.num_e: self.num_e + self.num_d]
    
    def _g_ntc(self):
        return self.sgRNAs[self.num_e + self.num_d: self.num_e + self.num_d + self.num_ntc]
    
    def _g_n(self):
        return self.sgRNAs[self.num_e + self.num_d + self.num_ntc: self.num_e + self.num_d + self.num_ntc + self.num_n]
    
    def _S(self):
        
        # currently different each time called, the S_l in dataframe doens't reflect the one used for treatments,
        # add in constructor?
        
        S = []
    
        g_e = self._g_e()
        g_d = self._g_d()
        g_ntc = self._g_ntc()
        g_n = self._g_n()

        for i in g_e:
            g_scalar = np.random.uniform(1.2, 2.0)
            for n in np.arange(i):
                S.append(g_scalar)

        for i in g_d:
            g_scalar = np.random.uniform(0.2, 1.0)
            for n in np.arange(i):
                S.append(g_scalar)
                
        for i in g_ntc:
            for n in np.arange(i):
                S.append(1)
                
        for i in g_n:
            for n in np.arange(i):
                S.append(1)
            
        return S 
    
    def _S_l(self):
        return np.multiply(self._S(), self.lam)
        
        
        
    def _type_of_change(self):
        """
        Sets number of enriched, depleted, NTC, and normal genes. 
        
        Returns
        -------
        type_of_change : list
            The list is filled with strings of enriched, depleted, NTC, 
            and normal for a gene. The number of times a type is in the 
            list is based on the fractional representation specified upon 
            initialization.
            
        """
        
        type_of_change = []
        
        g_e = self._g_e()
        g_d = self._g_d()
        g_ntc = self._g_ntc()
        g_n = self._g_n()
        
        e = ["enriched" for i in np.arange(len(g_e)) for n in np.arange(g_e[i])]
        d = ["depleted" for i in np.arange(len(g_d)) for n in np.arange(g_d[i])]
        ntc = ["ntc" for i in np.arange(len(g_ntc)) for n in np.arange(g_ntc[i])]
        n = ["normal" for i in np.arange(len(g_n)) for n in np.arange(g_n[i])]
        
        type_of_change = e + d + ntc + n
        
        return type_of_change 
    
    
    def sample(self):
        """
        Generates DataFrame with observations for the simulation. 
        
        Returns
        -------
        result : DataFrame 
            This DataFrame concatenates information from the _gene(),
            _num_sgRNAs(), _setting_treatment_libraries(),
            _setting_control_libraries(), _type_of_change() methods.  
            
        """
        
        # reorganize this to make code clearer
        sgRNA = pd.DataFrame({"sgRNAs": self._sgRNAs()})
        gene = pd.DataFrame({"gene": self._gene()})
        lam = pd.DataFrame({"lambda": self.lam})
        S_lam = pd.DataFrame({"modified lambda": self._S_l()})
        control = pd.DataFrame(self._setting_control_libraries()).T
        treatment = pd.DataFrame(self._setting_treatment_libraries()).T
        type_of_change = pd.DataFrame({"type": self._type_of_change()})
        
        result = pd.concat([sgRNA, gene, lam, S_lam, control, treatment, type_of_change], axis=1, join="inner")

        return result 

In [10]:
trial = Simulator(num_genes=5, num_treatment=1, num_control=1, fraction_depleted=0.2)
print(trial.sample())

      sgRNAs    gene     lambda  modified lambda       0       0      type
0    sgRNA_0  gene_0  24.345695        42.338360  4061.0  2182.0  enriched
1    sgRNA_1  gene_0  12.611626        21.932237  2030.0  1206.0  enriched
2    sgRNA_2  gene_0  18.899644        32.867409  3215.0  1608.0  enriched
3    sgRNA_3  gene_0  21.146372        36.774580  2707.0  1206.0  enriched
4    sgRNA_4  gene_0  10.792848        18.769293  2369.0   861.0  enriched
5    sgRNA_5  gene_1  29.941172        23.243749  5076.0  1780.0  depleted
6    sgRNA_6  gene_1  15.329538        11.900534  2030.0  1091.0  depleted
7    sgRNA_7  gene_1  18.876015        14.653713  3553.0  1034.0  depleted
8    sgRNA_8  gene_1  28.809071        22.364883  4230.0  1665.0  depleted
9    sgRNA_9  gene_1  17.354463        13.472512  2030.0  1034.0  depleted
10  sgRNA_10  gene_1  25.732208        19.976271  5414.0  1091.0  depleted
11  sgRNA_11  gene_2  16.394012        16.394012  3046.0  1091.0       ntc
12  sgRNA_12  gene_2  21.

In [4]:
# Figuring things out for the class 

sgRNAs = np.random.normal(loc=5, scale=1, size=5)
sgRNAs = np.round(sgRNAs)
print(sgRNAs)

l = [np.random.uniform(10, 30) for gene in sgRNAs for num in np.arange(gene)]
g = ["gene_" + str(i) for i in np.arange(len(sgRNAs)) for n in np.arange(sgRNAs[i])]

print(l)
print(len(l))
print(g)

# cuts count into half -- bc every count represents gene, so getting fraction wrt gene
g_e = sgRNAs[0: round(len(sgRNAs) * 0.5)] 
g_d = sgRNAs[round(len(sgRNAs) * 0.5): len(sgRNAs)]

print(g_e)
print(g_d) 

S = []

for i in g_e:
    g_scalar = np.random.uniform(1.2, 2.0)
    for n in np.arange(i):
        S.append(g_scalar)
        
for i in g_d:
    g_scalar = np.random.uniform(0.2, 1.0)
    for n in np.arange(i):
        S.append(g_scalar)


print(S)
l = np.array(l)
S = np.array(S)


S_l = np.multiply(S, l)
print(S_l)




# Do this for each library to fill the counts 
a = [np.random.poisson(i, size=1) for i in S_l]
a = np.concatenate(a)
a = a.astype(float)
a /= (a.sum())
a *= 100
a = np.round(a)

print(a)
print(a.sum())

# somewhere you should check that no count is negative 


e = ["enriched" for i in np.arange(len(g_e)) for n in np.arange(g_e[i])]
d = ["depleted" for i in np.arange(len(g_d)) for n in np.arange(g_d[i])]

[6. 4. 4. 5. 5.]
[18.2081412821021, 26.629510265094723, 22.997731577465526, 20.314316513787226, 28.55613380117996, 20.828848510005322, 10.886448516162055, 10.166226571719957, 28.695709724445717, 24.614395969609575, 10.121761903681607, 13.841134469328264, 17.4892144251403, 14.22473140420938, 15.474600693067348, 17.202360964112593, 13.575344096183846, 19.200165130848532, 16.805381442690877, 13.425824202161978, 17.78326475917534, 27.734371049800817, 23.11428840158324, 28.817907985480975]
24
['gene_0', 'gene_0', 'gene_0', 'gene_0', 'gene_0', 'gene_0', 'gene_1', 'gene_1', 'gene_1', 'gene_1', 'gene_2', 'gene_2', 'gene_2', 'gene_2', 'gene_3', 'gene_3', 'gene_3', 'gene_3', 'gene_3', 'gene_4', 'gene_4', 'gene_4', 'gene_4', 'gene_4']
[6. 4.]
[4. 5. 5.]
[1.7121905046452408, 1.7121905046452408, 1.7121905046452408, 1.7121905046452408, 1.7121905046452408, 1.7121905046452408, 1.9497738932495805, 1.9497738932495805, 1.9497738932495805, 1.9497738932495805, 0.23363755338401646, 0.23363755338401646, 0.23